In [17]:
#instantiate cursor and load libraries
import json
import pandas as pd
import requests
import mysql.connector
import config
import time
import urllib.request
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS

#### Helper Functions 

In [2]:
def get_ISBN(page):
    try:
        return page.find_all(class_='infoBoxRowItem')[1].get_text(strip=True).split('(')[0]
    except:
        return None

In [3]:
def get_title(page):
    try:
        return page.find(id = 'bookTitle').get_text(strip=True)
    except:
        return None

In [4]:
def get_author(page):
    try:
        return page.find('span', itemprop = 'name').get_text(strip=True)
    except:
        return None

In [5]:
def get_rating(page):
    try:
        return float(page.find('span', itemprop = 'ratingValue').get_text(strip=True))
    except:
        return None

In [6]:
def get_pages(page):
    try:
        return page.find('span', itemprop = 'numberOfPages').get_text(strip=True).split(' ')[0]
    except:
        return None

In [7]:
def get_publisher(page):
    try:
        details = page.find_all('div', 'row')
        row = details[1].get_text(strip=True)
        publisher = row.split('by')[1].strip()
        return publisher
    except:
        return None

In [8]:
def get_genre(page):
    try:
        genre = page.find_all('div', class_='elementList')[0].get_text(strip=True)
        genre = re.findall('(\w+[a-zA-Z])', genre)[0]
        return genre
    except: 
        return None

In [9]:
import re
def solve(s):                                             
    return re.sub(r'(\d)(st|nd|rd|th)', r'\1', s)

In [10]:
import datetime

In [11]:
def get_release_date(page):
    try:
        details = page.find_all('div', 'row')
        row = details[1].get_text(strip=True)
        row1 = row.split('Published')[1].strip()
        date = row1.split('by')[0].strip()
        date = datetime.datetime.strptime(solve(date), "%B %d %Y")
        return date
    except:
        return None

In [12]:
def get_series(page):
    try:
        if page.find_all('div', class_='infoBoxRowTitle')[3].get_text() == 'Series':
            return 1
        else:
            return 0
    except:
        return 0

In [13]:
def get_followers(page):
    try:
        bio = page.find('div', class_ = 'bookAuthorProfile')
        parse = bio.get_text(strip=True)
        parse1 = re.findall('(\d+(?:[\.,]\d+)?)',parse)[0]
        final = parse1.replace(',', '')
        return int(final)
    except:
        return None

# Scrape 2017

In [18]:
books_2017 = []
years = [2017]
genres = ['debut-goodreads-author', 'young-adult-fiction-books', 'young-adult-fantasy-books', 'childrens-books', 
          'picture-books','fiction-books', 'mystery-thriller-books', 'historical-fiction-books', 'fantasy-books', 
          'romance-books', 'science-fiction-books', 'horror-books', 'humor-books', 'nonfiction-books',
          'memoir-autobiography-books', 'history-biography-books', 'science-technology-books',
          'food-cookbooks', 'graphic-novels-comics', 'poetry-books']

def get_2017books():
    
    for year in years:
        
        print(year)
        
        for genre in genres:
            
            print(genre)
            
            url = f'https://www.goodreads.com/choiceawards/best-{genre}-{year}'
            resp = urllib.request.urlopen(url)
            soup = BS(resp, from_encoding=resp.info().get_param('charset'))
            
            for info in soup.find_all(href=True):
#                 print(info)

                if 'from_choice=true' in info['href']:
                    end_url = info['href']

                    each_book = {}

                    resp = urllib.request.urlopen(f'https://www.goodreads.com{end_url}')
                    content = BS(resp, 'html.parser')

                    each_book['title'] = get_title(content)
                    each_book['author'] = get_author(content)
                    each_book['rating'] = get_rating(content)
                    each_book['pages'] = get_pages(content)
                    each_book['publisher'] = get_publisher(content)
                    each_book['ISBN'] = get_ISBN(content)
                    each_book['genre'] = get_genre(content)
                    each_book['series'] = get_series(content)
                    each_book['author_followers'] = get_followers(content)
                    each_book['publish_date'] = get_release_date(content)

                    books_2017.append(each_book)

                    print(f"Inserted {each_book['title']}")

    pd.DataFrame(books_2017).to_csv('books_2017.csv')

    return books_2017
    

In [19]:
get_2017books()

2017
debut-goodreads-author
Inserted The Hate U Give
Inserted The Hate U Give
Inserted The Hate U Give
Inserted Caraval
Inserted The Bear and the Nightingale
Inserted We Were the Lucky Ones
Inserted Midnight at the Bright Ideas Bookstore
Inserted The Dry
Inserted American War
Inserted The Chilbury Ladies' Choir
Inserted My Absolute Darling
Inserted The Leavers
Inserted Stay with Me
Inserted All Our Wrong Todays
Inserted Goodbye, Vitamin
Inserted The Animators
Inserted I Liked My Life
Inserted What We Lose
Inserted Kings of the Wyld
Inserted The Coppersmith Farmhouse
Inserted The Idea of You
Inserted The Caledonian Gambit
young-adult-fiction-books
Inserted The Hate U Give
Inserted The Hate U Give
Inserted The Hate U Give
Inserted Turtles All the Way Down
Inserted One of Us Is Lying
Inserted The Gentleman's Guide to Vice and Virtue
Inserted Always and Forever, Lara Jean
Inserted Geekerella
Inserted Eliza and Her Monsters
Inserted Once and for All
Inserted When Dimple Met Rishi
Inserted B

Inserted Theft by Finding: Diaries 1977-2002
Inserted Believe Me: A Memoir of Love, Death, and Jazz Chickens
Inserted One Day We'll All Be Dead and None of This Will Matter
Inserted The Awkward Thoughts of W. Kamau Bell: Tales of a 6' 4", African American, Heterosexual, Cisgender, Left-Leaning, Asthmatic, Black and Proud Blerd, Mama's Boy, Dad, and Stand-Up Comedian
Inserted Al Franken: Giant of the Senate
Inserted I'm Fine...and Other Lies
Inserted Rhett & Link's Book of Mythicality: A Field Guide to Curiosity, Creativity, and Tomfoolery
Inserted Revolution for Dummies: Laughing Through the Arab Spring
Inserted Vacationland: True Stories from Painful Beaches
Inserted Stephen Colbert's Midnight Confessions
Inserted I Need a Lifeguard Everywhere But the Pool
Inserted This Is Really Happening
Inserted Everything Is Awful: And Other Observations
Inserted Waiting for the Punch: Words to Live by from the WTF Podcast
nonfiction-books
Inserted How to Be a Bawse: A Guide to Conquering Life
Ins

Inserted Big Mushy Happy Lump
Inserted Big Mushy Happy Lump
Inserted Big Mushy Happy Lump
Inserted Wonder Woman, Volume 1: The Lies
Inserted Saga, Vol. 7
Inserted Wires and Nerve
Inserted Monstress, Vol. 2: The Blood
Inserted Real Friends
Inserted Ms. Marvel, Vol. 6: Civil War II
Inserted Everyone's a Aliebn When Ur a Aliebn Too
Inserted The Best We Could Do
Inserted Paper Girls, Vol. 2
Inserted Band Together
Inserted My Favorite Thing Is Monsters, Vol. 1
Inserted The Legend of Korra: Turf Wars, Part One
Inserted The Vision, Volume 2: Little Better than a Beast
Inserted The Black Monday Murders, Vol. 1: All Hail, God Mammon
Inserted Black Hammer, Vol. 1: Secret Origins
Inserted Bitch Planet, Vol. 2: President Bitch
Inserted Imperial Phase, Part I
Inserted Username: Uprising
Inserted Spill Zone
poetry-books
Inserted The Sun and Her Flowers
Inserted The Sun and Her Flowers
Inserted The Sun and Her Flowers
Inserted Depression & Other Magic Tricks
Inserted Solo
Inserted Love Her Wild
Inser

[{'title': 'The Hate U Give',
  'author': 'Angie Thomas',
  'rating': 4.52,
  'pages': '464',
  'publisher': 'Balzer + Bray',
  'ISBN': '0062498533',
  'genre': 'Young',
  'series': 0,
  'author_followers': 11980,
  'publish_date': datetime.datetime(2017, 2, 28, 0, 0)},
 {'title': 'The Hate U Give',
  'author': 'Angie Thomas',
  'rating': 4.52,
  'pages': '464',
  'publisher': 'Balzer + Bray',
  'ISBN': '0062498533',
  'genre': 'Young',
  'series': 0,
  'author_followers': 11980,
  'publish_date': datetime.datetime(2017, 2, 28, 0, 0)},
 {'title': 'The Hate U Give',
  'author': 'Angie Thomas',
  'rating': 4.52,
  'pages': '464',
  'publisher': 'Balzer + Bray',
  'ISBN': '0062498533',
  'genre': 'Young',
  'series': 0,
  'author_followers': 11980,
  'publish_date': datetime.datetime(2017, 2, 28, 0, 0)},
 {'title': 'Caraval',
  'author': 'Stephanie Garber',
  'rating': 3.95,
  'pages': '407',
  'publisher': 'Flatiron Books',
  'ISBN': '1250095255',
  'genre': 'Fantasy',
  'series': 1,
  '